In [ ]:
%run ./01v_volve_generate_data.ipynb

import tensorflow as tf
import keras
from keras import layers
from keras import backend as K

import scipy.stats as stats
import math

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [ ]:
# expected input data shape: (batch_size, timesteps, data_dim)
max_epochs = 200
num_responses = 1
mini_batch_size = 5
num_features = 1

num_first_hidden_unit = 60
num_second_hidden_unit = 30

lstm_net = keras.models.Sequential()
lstm_net.add(layers.InputLayer(input_shape=(160, num_responses))) # (timesteps, data_dim)
lstm_net.add(layers.LSTM(num_first_hidden_unit,
                         batch_input_shape=(5, 160, num_responses),
                         return_sequences=True))
lstm_net.add(layers.Dropout(0.2))
lstm_net.add(layers.LSTM(num_second_hidden_unit,
                         return_sequences=True))
lstm_net.add(layers.Dropout(0.2))
lstm_net.add(layers.Dense(10))
lstm_net.add(layers.Dense(num_responses))

lstm_net.summary()

In [ ]:
def scheduler(epoch, lr):
    if epoch % 150 == 0 and epoch != 0:
        value = lr * tf.math.exp(-0.1)
        K.set_value(lstm_net.optimizer.lr, value)
        return K.get_value(lstm_net.optimizer.lr)
    else:
        return K.get_value(lstm_net.optimizer.lr)
    
lr_scheduler = keras.callbacks.LearningRateScheduler(scheduler, verbose=1)

optimizer = keras.optimizers.Adam(learning_rate=0.001, clipnorm=2)

metric = 'mae'

lstm_net.compile(optimizer=optimizer,
                 loss='mse',
                 metrics=metric)

history_lstm = lstm_net.fit(x_train,
                            y_train,
                            shuffle=False,
                            batch_size=mini_batch_size,
                            epochs=max_epochs,
#                            callbacks=[lr_scheduler],
                            validation_data=(x_valid, y_valid),
                            validation_freq=1)

In [ ]:
plt.plot(history_lstm.history[metric])
plt.show()

In [ ]:
y_predicted = lstm_net.predict(x_test, verbose=1)
# print(np.shape(y_predicted))

n = 3
plt.figure(figsize=(20, 10))
for i in range(1, n + 1):
    ax = plt.subplot(2, n, i)
    ax.set_title("y_validation {0}".format(i-1))
    plt.plot(y_test[i-1, :, 0])
    plt.plot(y_predicted[i-1, :, 0], '--r')
    plt.legend(['original Ip', 'predicted Ip'])

#     ax = plt.subplot(2, n, i + n, sharey=ax)
#     ax.set_title("y_predicted {0}".format(i-1))
#     plt.plot(y_predicted[i-1, :, 0])
plt.show()